In [ ]:
import os
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [ ]:
spark = SparkSession.builder \
    .appName("ner")\
    .master("local[1]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [ ]:
import time

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["document"])\
  .setOutputCol("token")

extractor = EntityExtractor()\
  .setEntities("file://" + os.getcwd() + "/entities.txt")\
  .setInputCols(["token", "sentence"])\
  .setOutputCol("entites")

finisher = Finisher() \
    .setInputCols(["entites"]) \
    .setIncludeKeys(False) \
    .setCleanAnnotations(True)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    extractor,
    finisher
  ])


In [ ]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("file://" + os.getcwd() + "../../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show(20)

In [ ]:
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")

In [ ]:
extracted = model.transform(data)
extracted.show()

In [ ]:
extracted.select("finished_entites")

In [ ]:
pipeline.write().overwrite().save("./extractor_pipeline")
model.write().overwrite().save("./extractor_model")

In [ ]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./extractor_pipeline")
sameModel = PipelineModel.read().load("./extractor_model")

sameModel.transform(data).show()